In [ ]:
import pandas as pd
from uszipcode import SearchEngine
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [ ]:
search = SearchEngine(simple_zipcode=False) # Gives a lot of interesting data if set to False
slalom_latitude = 33.854473
slalom_longitude = -84.360729

closest_zip_codes = search.by_coordinates(slalom_latitude, slalom_longitude, radius=10, returns=1000)
closest_zip_codes = list(map(lambda x: x.zipcode, closest_zip_codes))

print(len(closest_zip_codes))

In [ ]:
search

In [ ]:
df_2018 = pd.read_csv('Data/2018-12.csv')

In [ ]:
df_2018.head()

In [ ]:
# Clean up zip codes
df_2018['ZIP_CD'] = df_2018['ZIP_CD'].astype(str).replace('\.0', '', regex=True)

closest = df_2018[df_2018['ZIP_CD'].isin(closest_zip_codes)]
'''
Want Active Providers PGM_TRMNTN_CD == '0'
'''
closest[['FAC_NAME', 'PRVDR_CTGRY_CD','PGM_TRMNTN_CD', 'TRMNTN_EXPRTN_DT', 'CRTFD_BED_CNT']]

In [ ]:
print('number of zipcodes within 10 miles of Slalom: ' ,len(closest_zip_codes))
#print(closest[closest.duplicated(subset = ['FAC_NAME'])]['FAC_NAME'])
print('number of unique facilities within these zipcodes: ', len(closest['FAC_NAME'].unique()))
provider_status_counts_dict = closest['PGM_TRMNTN_CD'].value_counts().to_dict()
PGM_TRMNTN_CD_DICT = {
    0: 'ACTIVE PROVIDER',
    1: 'VOLUNTARY-MERGER, CLOSURE',
    4: 'VOLUNTARY-OTHER REASON FOR WITHDRAWAL',
    5: 'INVOLUNTARY-FAILURE TO MEET HEALTH/SAFETY REQ',
    7: 'OTHER-PROVIDER STATUS CHANGE'
}

print('\n# of Facilities by city')
fac_by_city_dict = closest['CITY_NAME'].value_counts().to_dict()
[print(k,':', v) for k,v in fac_by_city_dict.items()]

print('\nProvider status counts:')
[print(PGM_TRMNTN_CD_DICT[k],":", v) for k,v  in provider_status_counts_dict.items()]

bed_cnt_dict = {}
print('\nTOP 10 Largest Facilities by Bed Count')
for index, row in closest[['FAC_NAME','BED_CNT']].sort_values(by='BED_CNT', ascending=False)[:10].iterrows():
    print(row['FAC_NAME'], row['BED_CNT'])
    bed_cnt_dict[row['FAC_NAME']] = row['BED_CNT']

y_pos = np.arange(10)
plt.bar(y_pos, bed_cnt_dict.values(), align='center', alpha=0.5)
plt.xticks(y_pos, bed_cnt_dict.keys())
plt.xticks(rotation=90)
plt.ylabel('#')
plt.title('Top 10 largest Facilities by Bed Count')
plt.show()

print('\n# of times facility undergone change of ownership')
chow_cnt_dict = closest['CHOW_CNT'].value_counts().to_dict()
[print(k,':', v) for k,v in chow_cnt_dict.items()]

print('\nUrban or Rural')
ur_dict = closest['CBSA_URBN_RRL_IND'].value_counts().to_dict()
[print(k,':', v) for k,v in ur_dict.items()]

closest['ORGNL_PRTCPTN_DT'] = pd.to_datetime(closest['ORGNL_PRTCPTN_DT'], format='%Y%M%d')
closest['Age_of_facility'] = datetime.today().year - closest['ORGNL_PRTCPTN_DT'].dt.year
closest[['FAC_NAME', 'Age_of_facility']].sort_values(by='Age_of_facility', ascending=False)
facility_age_dict = dict(zip(closest.FAC_NAME, closest.Age_of_facility))

plt.hist(facility_age_dict.values(), 10)
plt.ylabel('# of Facilities')
plt.xlabel('Age')
plt.title('Facility Age')
plt.show()
   
    
    
    
    
    